In [ ]:
import pandas as pd
import numpy as np

Read the message data into 'df' DataFrame, label columns (The given dataset here doesn't exist anymore so replace it with any other dataset)

In [ ]:
df=pd.read_table("https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv",header=None)
names=['label','messages']
df.columns=names

Train-Test Split the message data using sklearn

X_train gives Training message data on which algorithm is trained on (excludes output or 'label' column)

X_test gives Testing message data on which algorithm is tested on (excludes output or 'label' column)

y_train gives 'label' column or output column for prediction derived from training data

y_train gives 'label' column or output column to be checked for prediction derived from testing data and calculate accuracy of algorithm

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['messages'], df['label'], test_size=0.30)
XTrain=X_train.reset_index(drop=True)
YTrain=y_train.reset_index(drop=True)
XTest=X_test.reset_index(drop=True)
YTest=y_test.reset_index(drop=True)

Count Vectorizer is called, Firstly the number of columns are decided based on number of disitnct words in the given data. 
Each row corresponds to a given message sentence and each column value will be the number of occurences of that distinct word (column name) in that sentence. This is all handled by CountVectorizer. 
 
CountVectorizer is internal to Jupyter Notebook, no need to import anything

The parameter to initialize CountVectorizer needs to be changed since it does not recognize single words like 'a' as distinct words. For this ' token_pattern=r"(?u)\b\w\w+\b" ' which is a default parameter is updated to ' token_pattern=r"(?u)\b\w+\b" ' which recognizes single words 

The vectorized data is converted to arrays for each message sentence

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(stop_words=None, token_pattern=r"(?u)\b\w+\b")
x=cv.fit_transform(XTrain)
a2=x.toarray()

Data from CountVectorizer a2 is sent to new DataFrame for training data which would be numerical now (no. of occurences of a distinct word in given sentence corresponding to a row) 

Column names are obtained from get_feature_names() function of CountVectorizer that gives names of distinct words

Output column appended

In [ ]:
dfTRAIN=pd.DataFrame(a2, columns =cv.get_feature_names())
dfTRAIN['Output']=YTrain

list1 contains column names of our training data (distinct words)

list0 contains different distinct values of output i.e spam and ham

Probability of <<'word'|'output value'>> has:
Numerator=(Total no. of occurences 'word' in sentences which are classified as given 'output value') + 1

Denomenator=(Total no. of occurences of all words in sentences which are classified as given 'output value') + (no. of distinct words in sentences which are classified as given 'output value')

P('word'|'output value')=Numerator/Denomenator

list2 has Total no. of occurences of all words in sentences which are classified as given 'output value'

Dict2 has keys 'wordOutput_value' and values as P(word|Output_value)

In [ ]:
Num=0

Dict1={}
Dict2={}

list1=dfTRAIN.columns
list0=dfTRAIN['Output'].value_counts().index

for i in range(0,len(list0)):
    for j in range(0,(len(list1)-1)):
        Num=dfTRAIN[dfTRAIN['Output']==list0[i]][list1[j]].sum()
        list2=dfTRAIN[dfTRAIN['Output']==list0[i]].sum()#Returns a series
        list2=list2.drop(labels=['Output'])

        Prob=(Num+1)/(list2.sum()+len(list2[list2>0]))

        print("P(Text=",list1[j],"|","Output=",list0[i],")=",Prob)
        
        Dict2[list1[j]+list0[i]]=Prob
        
print(Dict2)
        

In [ ]:
Test=pd.DataFrame(XTest, columns='Text')
Test['Class']=YTest

list3 is getting sidtinct words from each sentence of 'Test' data

list0 contains different distinct values of output i.e spam and ham

colnames are distinct words

Probability of a sentence in Test data having a certain 'Output Value' is calculated by multiplying each of their distinct words probabilities in given sentence for given 'Output Value'. These probabilities are fetched from Dict2 Dictionary.

This is compares with Probability of same sentence having another 'Output Value' in same way

All these probabilities of same sentence having different Output values is compared. Highest probability Output value for sentence gives classification of that sentence (meaning it has that Ouput value). This prediction is compared with Output Values of Test data to get final accuracy

In [ ]:
count=0

for k in range(0,Test.shape[0]):
    list3=(Test['Text'][k]).split(" ")
    colnames=list(dfTRAIN.columns)
    print(list3)

    Mul=1
    Greatest=0
    Greatest_Output='yo'

    for i in range(0,len(list0)):
        Mul=1
        for j in range(0,len(list3)):
            try:
                a=colnames.index(list3[j])
            except ValueError:
                print(list3[j])
                continue
            
            print("Dict:",Dict2[list3[j]+list0[i]])
            Mul=Dict2[list3[j]+list0[i]]*Mul
            print(Mul)
        Mul=Mul*(dfTRAIN[dfTRAIN['Output']==list0[i]].shape[0])/(dfTRAIN.shape[0])
      
            
        if(Mul>Greatest):
            Greatest=Mul
            print(Greatest)
            Greatest_Output=list0[i]
        
    print(Test['Text'][k],"is classified as:",Greatest_Output)
    if(Greatest_Output==Test['Class'][k]):
        count=count+1

print("Accuracy:",(count/Test.shape[0])) 
       